In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [ ]:
!wget https://github.com/TobiasGlaubach/python-ml-turbofan/archive/master.zip

In [ ]:
!unzip master.zip


In [ ]:
!ls

In [ ]:
path='/home/spark/shared/python-ml-turbofan-master/CMAPSSData'
print(path)


In [ ]:
os.listdir(path)

In [ ]:
train_files= [filename for filename in os.listdir(path) if(filename.startswith('train') and filename.endswith('.txt'))]
                                                           

In [ ]:
train_files[0]

In [ ]:
test_files= [filename for filename in os.listdir(path) if(filename.startswith('test') and filename.endswith('.txt'))]

In [ ]:
print(train_files)
test_files

In [ ]:
!ls

In [ ]:
train_rdd=sc.textFile(path+'/'+train_files[0])

In [ ]:
train_rdd=train_rdd.map(lambda line:line.split(" "))

In [ ]:
train_rdd.count()

In [ ]:
test_rdd=sc.textFile(path+'/'+test_files[0])
test_rdd=test_rdd.map(lambda line:line.split(" "))

In [ ]:
test_rdd.count()

In [ ]:
header = ["id", "cycle", "setting1", "setting2", "setting3",
                    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8",
                    "s9", "s10", "s11", "s12", "s13", "s14", "s15", 
                    "s16","s17", "s18", "s19", "s20", "s21"]

In [ ]:
train_df=train_rdd.toDF(header)

In [ ]:
train_df.collect()

In [ ]:
test_df=test_rdd.toDF(header)

In [ ]:
test_df.show()

In [ ]:
import pandas as pd
train_pandas_df=train_df.toPandas()

In [ ]:
train_pandas_df.head()

In [ ]:
test_pandas_df=test_df.toPandas()
test_pandas_df.head()

In [ ]:
train_pandas_df.isnull().sum()


In [ ]:
train_df.na.drop()

In [ ]:
test_df.na.drop()

In [ ]:
train_pandas_df.dropna()

In [ ]:
df = train_df.withColumn('id', train_df['id'].cast(IntegerType()))

In [ ]:
df.collect()

In [ ]:
def set_type(df,intlist=[],doublelist=[]):
    if len(intlist)>0:
        for col in intlist:
            df=df.withColumn(col,df[col].cast(IntegerType()))
                             
    if len(doublelist)>0:
        for col in doublelist:
            df=df.withColumn(col,df[col].cast(DoubleType()))
    return df                            
                             

In [ ]:
train_df.columns

In [ ]:
int_list = ["id", "cycle"]
double_list= list(set(train_df.columns)-set(int_list) )
double_list

In [ ]:
train_df=set_type(train_df,int_list,double_list)
test_df=set_type(test_df,int_list,double_list)

In [ ]:
train_df.columns

In [ ]:
test_df.schema

In [ ]:
train_df.schema

In [ ]:
print ("For train data:")
train_df.groupBy('id').count().sort(F.col("count")).show(1)


In [ ]:
train_df.groupBy('id').count().sort(F.col("count")).show(1)

In [ ]:
train_df.groupBy('id').count().sort(F.col("count").desc()).show(1)

In [ ]:
test_df.groupBy('id').count().show()

In [ ]:
train_df.toPandas().count()

# 2020-02-04 FEATURE SCALING

In [ ]:
rul=train_df.groupBy('id').agg({"cycle":"max"}).sort(("id"))

In [ ]:
type(rul)

In [ ]:
rul=rul.toDF('id','max')

In [ ]:
rul.show()

In [ ]:
temp_df=train_df.join(rul,on=['id'],how='left').toPandas()

In [ ]:
temp_df.sort_values(by=["id"])

In [ ]:
temp_df['RUL']=temp_df['max']-temp_df['cycle']

In [ ]:
temp_df.sort_values(by=["id","cycle"])

In [ ]:
def add_rul_labelling(df,id,time):
    rul=df.groupBy(id).agg({time:"max"}).sort('id')
    rul=rul.toDF('id','max')
    df=df.join(rul,on=['id'],how='left')
    df=df.withColumn("rul",df["max"]-df[time])
    return df

In [ ]:
df=add_rul_labelling(train_df,"id","cycle")

In [ ]:
df.toPandas()

In [ ]:
def add_window_labelling(df,w1,w2):
    df=df.withColumn('label1',F.when(df.rul<w1,1).otherwise(0))
    df=df.withColumn('label2',df.label1)
    df=df.withColumn('label2',F.when(df.rul<w2,2).otherwise(df.label1))
    return df

In [ ]:
train_df.collect()

In [ ]:
df=add_window_labelling(df,30,15)

In [ ]:
df.toPandas()

In [ ]:
path

In [ ]:
os.listdir(path)

In [ ]:
GT_FILE_TYPE="RUL"
gt_files=[filename for filename  in os.listdir(path) if filename.startswith(GT_FILE_TYPE) ]
gt_files

In [80]:
rdd_gt=sc.textFile(path+'/'+gt_files[0])
rdd_gt=rdd_gt.map(lambda line : line.split(" "))

In [83]:
rdd_gt.count()

100

In [100]:
header=["rul"]
df_gt=rdd_gt.toDF(header)

In [95]:
df_gt=df_gt.drop("_2")

In [98]:
df_gt.toPandas()

,rul
0,112
1,98
2,69
3,82
4,91
5,93
6,91
7,95
8,111
9,96


In [101]:
df_gt=df_gt.withColumn("rul",df_gt["rul"].cast(IntegerType()))

In [104]:
df_gt=df_gt.drop("_2")

In [105]:
df_gt

DataFrame[rul: int]